# Importing Libraires

In [ ]:
! pip install neurokit

In [1]:
! pip install -q -U tensorflow-addons

In [2]:
####### Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os 
import math
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.utils import shuffle
import tensorflow_addons as tfa
import gc

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu

# Data Loading

In [3]:
###### Dowloading Data
#! wget https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download
#! unzip './download'
# Clone the git repo
! git clone --quiet https://github.com/FedotovD/WESAD_ECG_SSL.git

In [5]:
###### Data Reading
def read_data(participants):
    # This function reads the data according to participants list taken as an argument

    X = []

    for filename in train_participants:
        temp = pd.read_csv('WESAD_ECG_SSL/data/{}.csv'.format(filename), index_col=0)
        X.append(temp)
    
    X = pd.concat(X, axis=0)
    y = X.lab.to_numpy()
    y = class_to_number(y)
    X = X.iloc[:,2:].to_numpy()
    
    return X, y

def class_to_number(labels):
    # This function converts classes from categories to numbers, e.g. "Base" to 0, "Fun" to 1, etc.

    classes = np.unique(labels)

    for c in range(len(classes)):
        labels[labels == classes[c]] = c
    
    return np.array(labels, dtype=int)

def one_hot_encoding(labels, num_classes):
    # This function performs one hot encoding for labels

    if type(labels[0]) == str:
        labels = class_to_number(labels)

    elif type(labels[0]) == float:
        labels = int(labels)

    Y = np.eye(num_classes)[labels]

    return Y

# Define train and test sets
train_participants = ['S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10']
devel_participants = ['S11', 'S13', 'S14']
test_participants  = ['S15', 'S16', 'S17']


# Read files for train and test participants
X_train, y_train = read_data(train_participants)
X_devel, y_devel = read_data(devel_participants)
X_test, y_test = read_data(test_participants)

# Normalize data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_devel_norm = scaler.transform(X_devel)
X_test_norm  = scaler.transform(X_test)

# Convert labels to one-hot
y_train_onehot = one_hot_encoding(y_train, 3)
y_devel_onehot = one_hot_encoding(y_devel, 3)
y_test_onehot = one_hot_encoding(y_test, 3)

In [6]:
###### Saving Numpy Arrays
np.savez_compressed('X_train_MHM_Stress.npz',np.array(X_train_norm))
np.savez_compressed('y_train_MHM_Stress.npz',np.array(y_train))
np.savez_compressed('X_dev_MHM_Stress.npz',np.array(X_test_norm))
np.savez_compressed('y_dev_MHM_Stress.npz',np.array(y_test)) 

In [7]:
###### Loading Dataset
X_train = np.array(np.load('./X_train_MHM_Stress.npz',allow_pickle=True)['arr_0'],dtype=np.float16)
X_train = np.reshape(X_train,(2019,2560,1))
X_dev = np.array(np.load('./X_dev_MHM_Stress.npz',allow_pickle=True)['arr_0'],dtype=np.float16)
X_dev = np.reshape(X_dev,(2019,2560,1))
y_train = np.load('./y_train_MHM_Stress.npz',allow_pickle=True)['arr_0']
y_dev = np.load('./y_dev_MHM_Stress.npz',allow_pickle=True)['arr_0']

###### Shuffling Numpy Arrays
X_train,y_train = shuffle(X_train,y_train)
X_dev,y_dev = shuffle(X_dev,y_dev)

print(X_train.shape)
print(X_dev.shape)
print(y_train.shape)
print(y_dev.shape)

In [ ]:
###### Signal Fourier Analysis

##### Function to Plot Frequency of Signal
def plot_fourier_spectrum(signal):
    
    """
    Funtion to plot Fourier Spectrum of any Bioradar Signal
    
    INPUTS:-
    1)signal : Normalized Signal in time domain
    
    """
    N = 200 # Signal Duration (Sampling Frequency)
    
    yf = fft(signal)
    xf = fftfreq(N, 1/100)
    
    plt.plot(sorted(xf), np.abs(yf))
    plt.grid('True')
    #plt.show()

##### Function to Plot Signal in Temporal Domain
def plot_time(signal):
    
    """
    Funtion to  Signal in Temporal Domain
    
    INPUTS:-
    1)signal : Normalized Signal in time domain
    
    """
    plt.plot(np.arange(2560),signal)
    plt.grid('True')
    
##### Fourier Spectrum Plotting
X_NRR = []
X_ARR = []

for i in range(200):
    if(y_train[i]==1):
        X_NRR.append(X_train[i])
    else:
        X_ARR.append(X_train[i])
        
for i in range(20):
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('-----------------------------------------------------')
    print('Fourier')
    #plot_fourier_spectrum(X_NRR[i])
    #plot_fourier_spectrum(X_ARR[i])
    plt.show()
    print('-----------------------------------------------------')
    print('Time')
    plot_time(X_NRR[i])
    #plot_time(X_ARR[i])
    plt.show()

# Model Making

## Transformer

In [9]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    pos_encoding = angle_rads[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
    # add extra dimensions to add the padding
    # to the attention logits. 
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def get_config(self):
        config = super(MultiHeadAttention, self).get_config().copy()
        config.update({
            'd_model': self.d_model,
            'num_heads':self.num_heads
        })
        
    def split_heads(self, x, batch_size):
        
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
               maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.maximum_position_encoding = maximum_position_encoding
        self.rate = rate

        #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def get_config(self):
        config = super(Encoder, self).get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads':self.num_heads,
            'dff':self.dff,
            'maximum_position_encoding':self.maximum_position_encoding,
            'rate':self.rate  
        })
        
    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        #x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)         

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        
    def get_config(self):
        config = super(EncoderLayer, self).get_config().copy()
        config.update({
            'd_model': self.d_model,
            'num_heads':self.num_heads,
            'dff':self.dff,
            'rate':self.rate  
        })
        
    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
 
        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
        return out2
    
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, 
                 pe_input, rate=0.1):
        super(Transformer, self).__init__()
        
        self.num_layers = num_layers
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.pe_input = pe_input
        self.rate = rate
        
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                                pe_input, rate)
        
    def get_config(self):
        config = super(Transformer,self).get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads':self.num_heads,
            'dff':self.dff,
            'pe_input':self.pe_input,
            'rate':self.rate  
        })
    
    def call(self, inp, training, enc_padding_mask):
        return self.encoder(inp, training, enc_padding_mask)

# Model Training

In [10]:
###### Defining Architecture

#with tpu_strategy.scope():

#### Defining Hyperparameters
num_layers = 2
d_model = 128
num_heads = 8
dff = 256
max_seq_len = 2560 #X_train.shape[1]
pe_input = 2560
rate = 0.3
num_features = 1

###### Defining Layers
Input_layer = tf.keras.layers.Input(shape=(max_seq_len,num_features))

##### Convolutional Filters

### Layer-1
conv1 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')
conv11 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')
conv12 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')
conv13 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')

### Layer-2
conv2 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')
conv21 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')
conv22 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')
conv23 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')

### Layer-3
conv3 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')
conv31 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')
conv32 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')
conv33 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')

#### Channel Attention Module
#cam_module = CAM_Module(128,1)

##### Transfromer Layer
#transformer = Transformer(num_layers,d_model,num_heads,dff,pe_input,rate)

##### Output Layer
gap_layer = tf.keras.layers.GlobalAveragePooling1D()

###### Defining Architecture
##### Input Layer
Inputs = Input_layer

##### Network
#### Layer-1
conv1_up = conv1(Inputs)
conv_11 = conv11(conv1_up) 
conv_12 = conv12(conv_11)
conv_13 = conv13(conv_12)
conv_13 = tf.keras.layers.Add()([conv_13,conv_11])

#### Layer-2
conv2_up = conv2(conv_13)
conv_21 = conv21(conv2_up)
conv_22 = conv22(conv_21)
conv_23 = conv23(conv_22)
conv_23 = tf.keras.layers.Add()([conv_23,conv_21])

#### Layer-3
conv3_up = conv3(conv_23)
conv_31 = conv31(conv3_up)
conv_32 = conv32(conv_31)
conv_33 = conv33(conv_32)
conv_33 = tf.keras.layers.Add()([conv_33,conv_31])

##### Transformer
#embeddings =  transformer(inp=conv_33,enc_padding_mask=None)

##### CAM Module
#cam_op = cam_module(conv_33)
#cam_op = tf.keras.layers.Add()([cam_op,embeddings])

##### Output Layers
#### Initial Layers
gap_op = gap_layer(conv_33)
dense1 = tf.keras.layers.Dense(256,activation='relu')(gap_op)
dropout1 = tf.keras.layers.Dropout(0.3)(dense1)
dense2 = tf.keras.layers.Dense(256,activation='relu')(dropout1)
dense3 = tf.keras.layers.Dense(3,activation='softmax')(dense2)

##### Compiling Architecture            
model = tf.keras.models.Model(inputs=Inputs,outputs=dense3)
model.load_weights('../input/mhm-stress-wesad-models/MHM_Stress_WESAD.h5')
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()      
tf.keras.utils.plot_model(model)

###### Model Training 

##### Model Checkpointing
filepath = './MHM_Stress_WESAD.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,monitor='val_accuracy',save_best_only=True,mode='max',save_weights_only=True)

##### Softmax Training 
#history = model.fit(X_train,y_train,epochs=100,batch_size=32,
#                validation_data=(X_dev,y_dev),validation_batch_size=32,
#               callbacks=checkpoint)

###### Plotting Metrics  
##### Accuracy and Loss Plots 

#### Accuracy
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('Model Accuracy')
#plt.ylabel('Accuracy')
#plt.xlabel('Epoch')  
#plt.legend(['Train', 'Testing'], loc='best')
#plt.grid(b='True',which='both')
#plt.show()

### Loss     
#plt.plot(history.history['loss'])  
#plt.plot(history.history['val_loss'])
#plt.title('Model Loss')  
#plt.ylabel('Loss')         
#plt.xlabel('epoch')
#plt.legend(['Train', 'Testing'], loc='best')
#plt.grid(b='True',which='both')
#plt.show()      

In [ ]:
###### Function to Plot Confusion Matrix
plt.rcParams["figure.figsize"] = [6,6]
import itertools
def plot_confusion_matrix(cm,classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
###### Confusion Matrix Computation
y_preds = model.predict(X_dev)
cm = confusion_matrix(y_dev,np.argmax(y_preds,axis=1))
print('//////////////////////////////////')
print(cm)
print('//////////////////////////////////')

###### Plotting Confusion Matix
cm_plot_labels = ['Normal','Arrhythmia']
plot_confusion_matrix(cm=cm,classes=cm_plot_labels,normalize=False,title='Confusion Matrix')
#plot_confusion_matrix(cm=cm,classes=cm_plot_labels,normalize=True,title='Confusion Matrix')

In [18]:
####### Model Testing

###### Testing Model - ArcFace Style                
    
def normalisation_layer(x):   
    return(tf.math.l2_normalize(x, axis=1, epsilon=1e-12))

predictive_model = tf.keras.models.Model(inputs=model.input,outputs=model.layers[-2].output)
predictive_model.compile(tf.keras.optimizers.Adam(lr=1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#y_in = tf.keras.layers.Input((11,))

Input_Layer_rdi = tf.keras.layers.Input((max_seq_len,num_features))
#Input_Layer_rai = tf.keras.layers.Input((T,H,W,C_rai))
op_1 = predictive_model(Input_Layer_rdi)

##Input_Layer_Flipped = tf.keras.layers.Input((224,224,3))
##op_2 = predictive_model([Input_Layer_Flipped,y_in]) 
##final_op = tf.keras.layers.Concatenate(axis=1)(op_1)

final_norm_op = tf.keras.layers.Lambda(normalisation_layer)(op_1)

testing_model = tf.keras.models.Model(inputs=Input_Layer_rdi,outputs=final_norm_op)
testing_model.compile(tf.keras.optimizers.Adam(lr=1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

##### Nearest Neighbor Classification
from sklearn.neighbors import KNeighborsClassifier
Test_Embeddings = testing_model.predict(X_dev)
Train_Embeddings = testing_model.predict(X_train)

col_mean = np.nanmean(Test_Embeddings, axis=0)
inds = np.where(np.isnan(Test_Embeddings))
#print(inds)
Test_Embeddings[inds] = np.take(col_mean, inds[1])

col_mean = np.nanmean(Train_Embeddings, axis=0)
inds = np.where(np.isnan(Train_Embeddings))
#print(inds)
Train_Embeddings[inds] = np.take(col_mean, inds[1])

###### t-SNE Plotting
import seaborn as sns
from sklearn.manifold import TSNE

#### Computing and Saving Output 
#np.savez_compressed('./Embeddings/RDCML_Softmax_Ablation.npz',Test_Embeddings)

#### t-SNE Plots
### t-SNE Embeddings
tsne_X_dev = TSNE(n_components=2,perplexity=30,learning_rate=10,n_iter=2000,n_iter_without_progress=50).fit_transform(Test_Embeddings) # t-SNE Plots 

### Plotting
plt.rcParams["figure.figsize"] = [12,8]
for idx,color_index in zip(list(np.arange(3)),sns.color_palette()):
    plt.scatter(tsne_X_dev[y_dev == idx, 0],tsne_X_dev[y_dev == idx, 1],s=40,color=color_index,edgecolors='k',marker='h')
plt.legend(['Baseline','Stressed','Amusement'],loc='best')
plt.grid(b='True',which='both')
plt.savefig('./MHM_Stress_WESAD_Softmax_Ablation.png')
plt.show()

In [ ]:
x = # Dimensions: (2560,1) Numpy Array 
x = (x - np.mean(x))/np.std(x)
x = np.reshape(x,(1,2560,1))

model.predict(x)